# TO-DO STEPS

Data Scraping:


## Data Processing (dla obu zbiorów danych)
1. Ułożenie danych w kolumnach.
2. Miejsca zerowe.
3. Feature Engineering. (kolumna koordynaty geograficzne, 


## Data analysis. (+potencjalnie Streamlit)
1. Prócz metrażu czynniki najbardziej wpływające na cenę.
2. Analiza ceny do piętra
3. Analiza odległości geograficznej od centrum
4. Analiza parking/balon
5. Analiza ceny/trendów w konkretnych dzielniach.


# Jeśli model to.
5. Onehotencoding.
6. Train/Test
7. Model.

Bonus points:
    wizualizacja geograficzna (streamlit - prettymap/mapbox)
    stosunek ilości balkonów w mieszkaniu na dzielnice (balkon rate)
    wpływ i częstotliwość '!' w nazwie oferty i ich zestawienie z ceną
    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [228]:
df_otodom = pd.read_csv('datasets/otodom_dataset_raw.csv')

In [229]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   6820 non-null   object
 1   name/title            6820 non-null   object
 2   address               6820 non-null   object
 3   price                 6820 non-null   object
 4   area                  6820 non-null   object
 5   price-per-area        6526 non-null   object
 6   floor/store           6720 non-null   object
 7   no of rooms           6820 non-null   object
 8   year of construction  6154 non-null   object
 9   parking space         4206 non-null   object
 10  market                6154 non-null   object
 11  form of ownership     4941 non-null   object
 12  condition             5229 non-null   object
 13  rent                  2846 non-null   object
 14  heating               4913 non-null   object
 15  advertiser type       6154 non-null   

## Braki w danych dotyczących ceny

### Wiersze bez podanej ceny

In [230]:
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_9756\765562517.py:1: SyntaxWarning: invalid escape sequence '\d'
  no_price = df_otodom[~df_otodom['price'].str.contains('\d')]


294

In [231]:
no_price

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
14,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 65m2 + balkon,"ul. Myśliwska, Płaszów, Podgórze, Kraków, mało...",Zapytaj o cenę,"65,56 m²",NaN,1/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
18,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 54m2 + balkon Bezpośrednio,"ul. Arsenał, Górka Narodowa, Prądnik Biały, Kr...",Zapytaj o cenę,"54,2 m²",NaN,5/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
40,https://www.otodom.pl/pl/oferta/emaus-garden-a...,Emaus Garden | apartament B2_A10,"ul. Piotra Borowego, Zwierzyniec, Zwierzyniec,...",Zapytaj o cenę,"56,43 m²",NaN,2/3,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
47,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 87m2 + balkon Bez Prowizji,"ul. Grzegórzecka 77, Grzegórzki, Grzegórzki, K...",Zapytaj o cenę,"87,95 m²",NaN,2/7,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
60,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 68m2 + loggia Bez Prowizji,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"68,59 m²",NaN,3/7,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 74m2 + loggia Bez Prowizji,"ul. Składowa, Krowodrza, Krowodrza, Kraków, ma...",Zapytaj o cenę,"74,31 m²",NaN,4/5,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6718,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 55m2 + 2 balkony,"ul. Bartosza Głowackiego, Bronowice, Bronowice...",Zapytaj o cenę,"55,08 m²",NaN,parter/4,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6735,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 73m2 + balkon Bezpośrednio,"Federowicza, Skotniki, Dębniki, Kraków, małopo...",Zapytaj o cenę,"73,11 m²",NaN,1/3,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6742,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 66m2 + 2 loggie,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"66,15 m²",NaN,6/7,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
# Usunięcie wierszy z brakującą ceną (no_price)
df_otodom.drop(no_price.index, inplace=True)

In [233]:
df_otodom['price'].count()

6526

In [234]:
# Sprawdzenie, czy wiersze zostały poprawnie usunięte
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_9756\4157452214.py:2: SyntaxWarning: invalid escape sequence '\d'
  no_price = df_otodom[~df_otodom['price'].str.contains('\d')]


0

### Ceny podane w innej walucie

In [235]:
#7 wierszy ma cenę w EUR
different_currency = df_otodom[~df_otodom['price'].str.contains('zł', na=False)]
different_currency['price'].count()

7

In [236]:
df_otodom.loc[different_currency.index]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
1265,https://www.otodom.pl/pl/oferta/penthouse-232m...,Penthouse 232m w zabytkowej kamienicy Stare Mi...,"Nowy Świat, Stare Miasto, Kraków, małopolskie",1 149 999 EUR,"232,1 m²",21 317 zł/m²,4/4,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,nie,NaN,kamienica,cegła
2056,https://www.otodom.pl/pl/oferta/penthouse-w-po...,Penthouse w pobliżu Rynku Głównego,"marszałka JózefaPiłsudskiego, Nowy Świat, Star...",1 150 000 EUR,232 m²,21 326 zł/m²,4/5,9,1906,NaN,wtórny,pełna własność,do zamieszkania,1 700 zł,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
3144,https://www.otodom.pl/pl/oferta/5-pokoi-taras-...,5 pokoi | Taras 77m2 | Komórka | Centrum | Mar...,"ul. marsz. Józefa Piłsudskiego, Nowy Świat, St...",1 149 999 EUR,"232,1 m²",21 317 zł/m²,4/4,5,1908,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,850 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 535 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4722,https://www.otodom.pl/pl/oferta/penthouse-tara...,Penthouse tarasem na dachu w kamienicy z windą,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"228,7 m²",21 634 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,NaN,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 700 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,220 EUR,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,cegła
4995,https://www.otodom.pl/pl/oferta/penthouse-star...,Penthouse Stare Miasto II Poziomy. Taras 70m2,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"232,1 m²",21 317 zł/m²,4/5,5,1906,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji


In [237]:
# zamiana cen na wartości liczbowe
df_otodom['price'] = df_otodom['price'].str.replace(' ', '')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_9756\4193540164.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)


In [238]:
df_otodom.loc[different_currency.index, 'price']

1265    1149999.0
2056    1150000.0
3144    1149999.0
3977     585280.0
4722    1150000.0
4908    1150000.0
4995    1150000.0
Name: price, dtype: float64

In [239]:
# kurs na 5.04.2024
exchange_rate = 4.28

df_otodom.loc[different_currency.index, 'price'] *= exchange_rate

In [240]:
df_otodom.loc[different_currency.index, 'price']

1265    4921995.72
2056    4922000.00
3144    4921995.72
3977    2504998.40
4722    4922000.00
4908    4922000.00
4995    4922000.00
Name: price, dtype: float64

In [241]:
df_otodom['price'].count()

6526

### Zamiana wartości w kolumnach na typ liczbowy

In [242]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

In [243]:
df_otodom['area']

0       55.98
1       87.00
2       47.10
3       66.69
4       65.81
        ...  
6815    66.86
6816    67.56
6817    47.35
6818    36.09
6819    73.74
Name: area, Length: 6526, dtype: float64

In [244]:
df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False) & ~df_otodom['price-per-area'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material


In [245]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

In [246]:
#trzy wiersze w innej walucie
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",2504998.4,91.45,27535.0,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",4922000.0,228.00,21700.0,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,220 EUR,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,cegła
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620000.0,35.55,17440.0,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [247]:
eur_rent_price = df_otodom[df_otodom['rent'].str.contains('EUR', na=False)]
gbp_rent_price = df_otodom[df_otodom['rent'].str.contains('GBP', na=False)]

In [248]:
# zamiana wartości rent na typ liczbowy
df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_9756\1831358940.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)


In [249]:
euro_exchange_rate = 4.28
gbp_exchange_rate = 4.99

df_otodom.loc[eur_rent_price.index, 'rent'] *= euro_exchange_rate
df_otodom.loc[gbp_rent_price.index, 'rent'] *= gbp_exchange_rate

In [250]:
df_otodom.loc[eur_rent_price.index, 'rent']

3977    428.0
4908    941.6
Name: rent, dtype: float64

In [251]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_9756\844144124.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)


In [252]:
# Rozdzielenie kolumny 'floor/store' na dwie osobne kolumny
df_otodom[['floor', 'store']] = df_otodom['floor/store'].str.split('/', expand=True)

# Usunięcie oryginalnej kolumny 'floor/store'
df_otodom.drop(columns=['floor/store'], inplace=True)


In [253]:
df_otodom[df_otodom['store'].isnull()]

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
42,https://www.otodom.pl/pl/oferta/ok-lasu-borkow...,"Ok. Lasu Borkow. piękne, duże mieszkanie, 2010r.","ul. Gwieździsta, Ruczaj, Dębniki, Kraków, mało...",950000.0,70.00,13571.0,3,2010,NaN,wtórny,...,do zamieszkania,450.0,gazowe,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
49,https://www.otodom.pl/pl/oferta/pieciopokojowe...,Pięciopokojowe 116m2 bez 2% Pcc,"ul. Miarowa, Swoszowice, Swoszowice, Kraków, m...",1300000.0,116.00,11207.0,5,2020,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,taras,blok,brak informacji,NaN,NaN
91,https://www.otodom.pl/pl/oferta/dom-z-ogrodkie...,Dom z ogródkiem | Dwa m. postojowe | gotowe,"Prądnik Biały, Prądnik Biały, Kraków, małopolskie",759000.0,76.70,9896.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,pustak,NaN,NaN
96,https://www.otodom.pl/pl/oferta/okazja-ladne-m...,Okazja ! Ładne Mieszkanie Bezczynszowe,"Bronowice Wielkie, Prądnik Biały, Kraków, mało...",490000.0,34.00,14412.0,2,brak informacji,garaż/miejsce parkingowe,wtórny,...,do wykończenia,NaN,elektryczne,prywatny,nie,ogródek,szeregowiec,cegła,parter,None
126,https://www.otodom.pl/pl/oferta/jasne-mieszkan...,Jasne mieszkanie 44 m2 przy ul. Wrocławskiej 53,"ul. Wrocławska, Krowodrza, Krowodrza, Kraków, ...",875000.0,44.00,19886.0,2,brak informacji,garaż/miejsce parkingowe,wtórny,...,do zamieszkania,700.0,miejskie,biuro nieruchomości,tak,balkon,brak informacji,brak informacji,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,https://www.otodom.pl/pl/oferta/nowa-huta-3-po...,"Nowa Huta, 3 pokoje, po remoncie 57m","Nowa Huta, Nowa Huta, Kraków, małopolskie",799000.0,57.00,14018.0,3,1952,NaN,wtórny,...,do zamieszkania,600.0,miejskie,biuro nieruchomości,nie,NaN,blok,cegła,3,None
6694,https://www.otodom.pl/pl/oferta/przestronne-mi...,Przestronne mieszkanie,"ul. Stańczyka, Bronowice, Bronowice, Kraków, m...",680000.0,36.56,18600.0,2,2020,NaN,wtórny,...,do zamieszkania,490.0,NaN,prywatny,tak,balkon,blok,brak informacji,2,None
6706,https://www.otodom.pl/pl/oferta/nowy-apartamen...,NOWy apartament z dużym ogrodem Olszanica,"ul. Olszanicka, Chełm, Zwierzyniec, Kraków, ma...",1195000.0,111.00,10766.0,5,2024,NaN,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,"balkon, ogródek",apartamentowiec,pustak,NaN,NaN
6715,https://www.otodom.pl/pl/oferta/krakow-ksiecia...,Kraków Księcia Józefa mieszkanie,"ul. Astronautów, Bielany, Zwierzyniec, Kraków,...",485000.0,39.00,12436.0,1,brak informacji,NaN,wtórny,...,do zamieszkania,270.0,gazowe,prywatny,nie,NaN,blok,cegła,2,None


In [254]:
df_otodom[df_otodom['floor'].isnull()]

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
42,https://www.otodom.pl/pl/oferta/ok-lasu-borkow...,"Ok. Lasu Borkow. piękne, duże mieszkanie, 2010r.","ul. Gwieździsta, Ruczaj, Dębniki, Kraków, mało...",950000.0,70.00,13571.0,3,2010,NaN,wtórny,...,do zamieszkania,450.0,gazowe,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
49,https://www.otodom.pl/pl/oferta/pieciopokojowe...,Pięciopokojowe 116m2 bez 2% Pcc,"ul. Miarowa, Swoszowice, Swoszowice, Kraków, m...",1300000.0,116.00,11207.0,5,2020,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,taras,blok,brak informacji,NaN,NaN
91,https://www.otodom.pl/pl/oferta/dom-z-ogrodkie...,Dom z ogródkiem | Dwa m. postojowe | gotowe,"Prądnik Biały, Prądnik Biały, Kraków, małopolskie",759000.0,76.70,9896.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,pustak,NaN,NaN
144,https://www.otodom.pl/pl/oferta/piekne-mieszka...,"Piękne mieszkanie dla rodziny z ogródkiem, 202...","Kobierzyn, Dębniki, Kraków, małopolskie",820000.0,64.00,12813.0,3,2020,NaN,wtórny,...,do zamieszkania,700.0,miejskie,biuro nieruchomości,nie,ogródek,blok,brak informacji,NaN,NaN
221,https://www.otodom.pl/pl/oferta/mieszkanie-z-o...,"Mieszkanie z ogródkiem w Swoszowicach - 78,45 m2","Swoszowice, Swoszowice, Kraków, małopolskie",749000.0,78.45,9547.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,silikat,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,https://www.otodom.pl/pl/oferta/51m2-nowa-huta...,"51m2, Nowa Huta, Ogródek, Po generalnych remoncie","os. Krakowiaków, Nowa Huta, Nowa Huta, Kraków,...",790000.0,50.58,15619.0,2,1955,NaN,wtórny,...,do zamieszkania,750.0,miejskie,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
6418,https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,"Mieszkanie 2-pokojowe | 35,8 m2 | Azory","Azory, Prądnik Biały, Kraków, małopolskie",536000.0,35.80,14972.0,2,1977,NaN,wtórny,...,NaN,457.0,miejskie,biuro nieruchomości,nie,NaN,blok,wielka płyta,NaN,NaN
6439,https://www.otodom.pl/pl/oferta/3-pok-kuchnia-...,"3 pok.+ kuchnia, udziały w nieruchomości, Pilotów","ul. Widna, Rakowice, Prądnik Czerwony, Kraków,...",365000.0,65.00,5615.0,3,1960,NaN,wtórny,...,do zamieszkania,30.0,gazowe,biuro nieruchomości,nie,ogródek,brak informacji,brak informacji,NaN,NaN
6706,https://www.otodom.pl/pl/oferta/nowy-apartamen...,NOWy apartament z dużym ogrodem Olszanica,"ul. Olszanicka, Chełm, Zwierzyniec, Kraków, ma...",1195000.0,111.00,10766.0,5,2024,NaN,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,"balkon, ogródek",apartamentowiec,pustak,NaN,NaN


In [255]:
df_otodom = df_otodom.dropna(subset=['floor', 'store'])

In [256]:
df_otodom.head()

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719000.0,55.98,12844.0,3,2022,NaN,pierwotny,...,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,ogródek,apartamentowiec,brak informacji,parter,8
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1350000.0,87.00,15517.0,4,2021,garaż/miejsce parkingowe,wtórny,...,NaN,900.0,miejskie,biuro nieruchomości,tak,balkon,brak informacji,cegła,2,4
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599000.0,47.10,12718.0,3,2023,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji,1,4
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833500.0,66.69,12498.0,3,2024,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji,3,8
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949000.0,65.81,14420.0,3,2023,NaN,wtórny,...,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji,7,7


In [257]:
# Zamiana wartości "parter" na 0 w kolumnie "floor"
df_otodom['floor'] = df_otodom['floor'].replace('parter', 0)

In [258]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor"
df_otodom['floor'] = df_otodom['floor'].replace('suterena', -1)

In [260]:
# Konwersja kolumny "floor" na typ liczbowy, zamieniając nieliczbowe wartości na NaN
df_otodom['floor'] = pd.to_numeric(df_otodom['floor'], errors='coerce')

# Usunięcie wierszy zawierających wartości NaN w kolumnie "floor"
df_otodom = df_otodom[~df_otodom['floor'].isna()]

In [261]:
df_otodom['floor'] = df_otodom['floor'].astype(int)

In [263]:
df_otodom['store'] = df_otodom['store'].astype(int)

In [264]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6112 entries, 0 to 6819
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   6112 non-null   object 
 1   name/title            6112 non-null   object 
 2   address               6112 non-null   object 
 3   price                 6112 non-null   float64
 4   area                  6112 non-null   float64
 5   price-per-area        6112 non-null   float64
 6   no of rooms           6112 non-null   int32  
 7   year of construction  5704 non-null   object 
 8   parking space         3681 non-null   object 
 9   market                5704 non-null   object 
 10  form of ownership     4372 non-null   object 
 11  condition             4639 non-null   object 
 12  rent                  2697 non-null   float64
 13  heating               4567 non-null   object 
 14  advertiser type       5704 non-null   object 
 15  elevator              5704

In [265]:
df_otodom['price'].mean()

985104.6141753927

In [266]:
df_otodom.describe()

,price,area,price-per-area,no of rooms,rent,floor,store
count,6.112000e+03,6112.000000,6112.000000,6112.000000,2697.000000,6112.000000,6112.000000
mean,9.851046e+05,59.900411,16574.702716,2.698135,480.097293,2.282886,4.675393
std,6.599385e+05,32.705208,4520.633846,1.059980,235.621399,2.008813,2.577294
min,9.500000e+04,10.270000,271.000000,1.000000,0.000000,-1.000000,1.000000
25%,6.650000e+05,41.000000,13749.750000,2.000000,350.000000,1.000000,3.000000
50%,8.344000e+05,53.000000,15984.000000,3.000000,500.000000,2.000000,4.000000
75%,1.080000e+06,68.790000,18500.000000,3.000000,650.000000,3.000000,6.000000
max,1.350000e+07,706.000000,70000.000000,10.000000,2095.800000,10.000000,93.000000


In [19]:
df_nieruchomosci = pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')

In [20]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3949 entries, 0 to 3948
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  3949 non-null   object 
 1   name/title                           3949 non-null   object 
 2   address                              3848 non-null   object 
 3   price                                3920 non-null   object 
 4   area                                 3949 non-null   object 
 5   price-per-area                       3920 non-null   object 
 6   floor/store                          3949 non-null   object 
 7   no of floors/stores in the building  3710 non-null   float64
 8   no of rooms                          3949 non-null   object 
 9   year of construction                 3949 non-null   object 
 10  parking space                        3949 non-null   object 
 11  market                        

In [21]:
df_nieruchomosci.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899 000 zł,"51,70 m²","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1 575 000 zł,125 m²,12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1 250 000 zł,"65,23 m²","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740 250 zł,"49,35 m²",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1 200 000 zł,"71,06 m²","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"


In [22]:
#9 cen jest w innej walucie
df_nieruchomosci[df_nieruchomosci['price'].str.contains(r'\d', na=False) & ~df_nieruchomosci['price'].str.contains('zł', na=False)].count()

url                                    9
name/title                             9
address                                9
price                                  9
area                                   9
price-per-area                         9
floor/store                            9
no of floors/stores in the building    9
no of rooms                            9
year of construction                   9
parking space                          9
market                                 9
form of ownership                      1
dtype: int64

In [23]:
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '')
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ksobc\AppData\Local\Temp\ipykernel_18712\2928192372.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['price'] = df_nieruchomosci['price'].str.extract('(\d+)').astype(float)


In [24]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '')
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(',', '.').astype(float)

In [25]:
#9 cen jest w innej walucie
df_nieruchomosci[df_nieruchomosci['price-per-area'].str.contains(r'\d', na=False) & ~df_nieruchomosci['price-per-area'].str.contains('zł', na=False)].count()

url                                    9
name/title                             9
address                                9
price                                  9
area                                   9
price-per-area                         9
floor/store                            9
no of floors/stores in the building    9
no of rooms                            9
year of construction                   9
parking space                          9
market                                 9
form of ownership                      1
dtype: int64

In [26]:
df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '')
df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ksobc\AppData\Local\Temp\ipykernel_18712\1934218877.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.extract('(\d+)').astype(float)


In [27]:
#niektóre mieszkania zamiast liczby pokoi mają wpisane "parter/suterena/-"?
df_nieruchomosci[~df_nieruchomosci['no of rooms'].str.contains(r'\d')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
37,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1373420.0,79.85,17200.0,II kwartał 2024,NaN,parter,4,-,II kwartał 2024,pierwotny (oferta dewelopera)
45,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,NaN,58.36,NaN,III kwartał 2024,NaN,suterena,3,garaż,dostępne,NaN
71,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Bronowice Wielkie, ul. Piaskow...",NaN,903189.0,73.43,12300.0,oddana do użytku,NaN,parter,3,naziemne,"IV kwartał 2022, oddana do użytku",pierwotny (oferta dewelopera)
119,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wola Justowska, ul. Królowej J...",NaN,1221192.0,72.69,16800.0,zrealizowana,NaN,parter,3,garaż,"październik 2023, zrealizowana",pierwotny (oferta dewelopera)
178,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Tonie, ul. Władysława Łokietka",NaN,1638000.0,136.60,11991.0,październik 2025,NaN,parter,4,garaż,październik 2025,pierwotny (oferta dewelopera)
212,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prądnik Biały, Piaszczysta/Pęk...",NaN,917868.0,62.44,14700.0,IV kwartał 2024,NaN,parter,3,garaż,IV kwartał 2024,pierwotny (oferta dewelopera)
221,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,NaN,55.00,NaN,wrzesień 2024,NaN,parter,3,naziemne,zarezerwowane,NaN
266,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Krowodrza Górka, ul. Józefa Wy...",NaN,1130000.0,72.34,15620.0,I kwartał 2024,NaN,parter,3,garaż,I kwartał 2024,pierwotny (oferta dewelopera)
268,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Krowodrza Górka, ul. Józefa Wy...",NaN,680000.0,38.88,17489.0,I kwartał 2024,NaN,parter,2,garaż,I kwartał 2024,pierwotny (oferta dewelopera)
274,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Dziekanowice, ul. Piasta Kołod...",NaN,NaN,45.01,NaN,III kwartał 2025,NaN,parter,3,garaż,dostępne,NaN


In [28]:
#df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.replace(' ', '')
#df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].str.extract('(\d+)').astype(int)

In [29]:
df_nieruchomosci.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899000.0,51.70,17388.0,1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1575000.0,125.00,12600.0,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1250000.0,65.23,19162.0,2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740250.0,49.35,15000.0,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.0,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"


In [30]:
df_nieruchomosci.describe()

,price,area,price-per-area,no of floors/stores in the building
count,3.920000e+03,3949.000000,3920.000000,3710.000000
mean,1.023459e+06,62.012249,16783.809439,4.850135
std,7.258431e+05,36.452915,4992.176616,2.531264
min,1.030000e+05,12.300000,1816.000000,1.000000
25%,6.800000e+05,41.760000,13802.500000,3.000000
50%,8.406720e+05,54.770000,16072.500000,4.000000
75%,1.116449e+06,70.000000,18926.000000,6.000000
max,1.600000e+07,795.000000,71018.000000,45.000000


In [31]:
df_nieruchomosci.isnull().sum()

url                                       0
name/title                                0
address                                 101
price                                    29
area                                      0
price-per-area                           29
floor/store                               0
no of floors/stores in the building     239
no of rooms                               0
year of construction                      0
parking space                             0
market                                   31
form of ownership                      2461
dtype: int64

In [32]:
df_nieruchomosci.shape

(3949, 13)